# Training totals

This notebook explores my training for DJBB 2024 in Jamaica.

## Setup imports

In [56]:
%matplotlib inline

import os, sys
import shutil
import tempfile
import glob
import re
import time
import datetime
import copy
import pickle
import hashlib
import logging
import warnings
import random

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style("whitegrid")
pd.set_option('display.max_rows', 100)

In [57]:
# log level
logging.basicConfig(
    stream=sys.stdout,
    format='%(asctime)s %(levelname)s: %(message)s',
    level=logging.INFO)

logger = logging.getLogger("notebook")
logger.setLevel(logging.DEBUG)

# disable font not found warnings
logging.getLogger('matplotlib.font_manager').disabled = True

## Setup data directory

In [58]:
# Use Jupyter magic to retain variable in this notebook
%env DATA_DIRECTORY=data
dataset_name = "my_activity_data=20240508102713.csv"
directory = os.environ.get("DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
data_dir = os.path.join(root_dir, dataset_name)
print(f"Data root: {root_dir}, file: {data_dir}")

env: DATA_DIRECTORY=data
Data root: data, file: data/my_activity_data=20240508102713.csv


In [59]:
df = pd.read_csv(data_dir, parse_dates=['start_date', 'start_date_local'])

## Parse data dump

In [78]:
# 2024 training data
training = df.loc[(df.start_date > '2024-01-01') & (df.start_date_local < '2024-04-30')]

In [86]:
# DJBB 2024
race = df.loc[df.name.str.contains('DJBB')]

In [91]:
training.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,pr_count,total_photo_count,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_temp
8,2,Morning Ride,4780.4,987,1914,61.0,Ride,Ride,NaN,11291134982,...,0,0,False,NaN,104560661,1,a11291134982,ehcmBr~ssMv@X|@b@~CbA`@TFNCHqAhC[|@uBvE]`@a@Zm...,2,32.0
9,2,Morning Ride,49279.1,10733,17090,800.0,Ride,Ride,10.0,11284528671,...,19,1,False,61.0,104560661,1,a11284528671,ghcmBx~ssMbIlCjFeAC\xc@|RlEpDjO|TxBkAxFYhFqC`P...,2,26.0
10,2,Morning Ride,30850.8,6768,8325,694.0,Ride,Ride,10.0,11267859653,...,14,4,False,59.0,104560661,1,a11267859653,oycmBprrsM|@gAv@{C~@o@zAM|@JvGrDtBb@zO?tD~AvCV...,2,23.0
11,2,Lunch Swim,475.0,1807,1807,0.0,Swim,Swim,NaN,11262576995,...,0,2,False,12.0,104560661,1,a11262576995,NaN,2,NaN
12,2,Lunch Ride,6438.6,1353,8077,103.0,Ride,Ride,12.0,11263227838,...,0,0,False,NaN,104560661,1,a11263227838,ihcmBz~ssM~Al@hBj@bA^PJRD\AhAc@^ILIDQB]FmABKBE...,2,38.0


## Training statistics

In [136]:
from dateutil import relativedelta
 
def convert_seconds(n):
    rd = relativedelta.relativedelta(seconds=n)
    return "{}h {:02d}m".format(rd.days*24+rd.hours, rd.minutes)

In [137]:
# total training moving time
convert_seconds(training.moving_time.sum())

'113h 11m'

In [138]:
# for each sport
sport_types = list(training.type.unique())

for s in sport_types:
    total = training.loc[training.type == s].moving_time.sum()
    pretty_time = convert_seconds(total)
    print("{}: {}".format(s,pretty_time))

Ride: 60h 35m
Swim: 4h 13m
Walk: 11h 26m
Run: 4h 58m
Workout: 8h 27m
VirtualRide: 19h 13m
WeightTraining: 4h 16m


In [189]:
# riding hours and kilometers
ride_df =  training.loc[training.type == 'Ride']

total_km = ride_df.distance.sum() / 1000
active_wks = (17-8-2+1)
km_per_wk = total_km / active_wks

total_elevation_gain = ride_df.total_elevation_gain.sum()

print("Active riding weeks:", active_wks)
print("Average rides per week: {:.1f}".format(ride_df.shape[0]/active_wks))
print("Average km per week: {:.1f}".format(km_per_wk))
print("Total km: {:.1f}".format(total_km))
print("Total elevation gain (m): {:.0f}\n".format(total_elevation_gain))

Active riding weeks: 8
Average rides per week: 4.0
Average km per week: 166.1
Total km: 1328.9
Total elevation gain (m): 12106



In [182]:
# temperatures
ride_df.average_temp.describe()

count    32.000000
mean     26.687500
std       5.396758
min      19.000000
25%      23.000000
50%      25.500000
75%      30.250000
max      39.000000
Name: average_temp, dtype: float64

## DJBB2024 statistics

In [159]:
# total moving time
convert_seconds(race.moving_time.sum())

'26h 10m'

In [160]:
total_dist_km = race.distance.sum()/1000
print("Total distance: {:.1f} ({:.1f}miles)".format(total_dist_km, total_dist_km/1.6))

Total distance: 627.4 (392.1miles)


In [161]:
avg_spd_km = total_dist_km / (race.moving_time.sum()/3600)
print("Average speed: {:.1f}km ({:.1f}mph)".format(avg_spd_km, avg_spd_km/1.6))

Average speed: 24.0km (15.0mph)


In [196]:
total_elevation_gain_m = race.total_elevation_gain.sum()
print("Total elevation gain: {:.1f}m ({:.1f}feet)".format(
    total_elevation_gain_m,
    total_elevation_gain_m/.3048))

Total elevation gain: 5983.0m (19629.3feet)


In [183]:
# temperatures
race.average_temp.describe()

count     5.00000
mean     30.60000
std       1.67332
min      28.00000
25%      30.00000
50%      31.00000
75%      32.00000
max      32.00000
Name: average_temp, dtype: float64